# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

---

## <font face="仿宋">第四部分导读

&emsp;&emsp;<font face="仿宋">在案例的第二、三部分中，我们详细介绍了关于特征工程的各项技术，特征工程技术按照大类来分可以分为数据预处理、特征衍生、特征筛选三部分，其中特征预处理的目的是为了将数据集整理、清洗到可以建模的程度，具体技术包括缺失值处理、异常值处理、数据重编码等，是建模之前必须对数据进行的处理和操作；而特征衍生和特征筛选则更像是一类优化手段，能够帮助模型突破当前数据集建模的效果上界。并且我们在第二部分完整详细的介绍机器学习可解释性模型的训练、优化和解释方法，也就是逻辑回归和决策树模型。并且此前我们也一直以这两种算法为主，来进行各个部分的模型测试。

&emsp;&emsp;<font face="仿宋">而第四部分，我们将开始介绍集成学习的训练和优化的实战技巧，尽管从可解释性角度来说，集成学习的可解释性并不如逻辑回归和决策树，但在大多数建模场景下，集成学习都将获得一个更好的预测结果，这也是目前效果优先的建模场景下最常使用的算法。

&emsp;&emsp;<font face="仿宋">总的来说，本部分内容只有一个目标，那就是借助各类优化方法，抵达每个主流集成学习的效果上界。换而言之，本部分我们将围绕单模优化策略展开详细的探讨，涉及到的具体集成学习包括随机森林、XGBoost、LightGBM、和CatBoost等目前最主流的集成学习算法，而具体的优化策略则包括超参数优化器的使用、特征衍生和筛选方法的使用、单模型自融合方法的使用，这些优化方法也是截至目前，提升单模效果最前沿、最有效、同时也是最复杂的方法。其中有很多较为艰深的理论，也有很多是经验之谈，但无论如何，我们希望能够围绕当前数据集，让每个集成学习算法优化到极限。值得注意的是，在这个过程中，我们会将此前介绍的特征衍生和特征筛选视作是一种模型优化方法，衍生和筛选的效果，一律以模型的最终结果来进行评定。而围绕集成学习进行海量特征衍生和筛选，也才是特征衍生和筛选技术能发挥巨大价值的主战场。

&emsp;&emsp;<font face="仿宋">而在抵达了单模的极限后，我们就会进入到下一阶段，也就是模型融合阶段。需要知道的是，只有单模的效果到达了极限，进一步的多模型融合、甚至多层融合，才是有意义的，才是有效果的。

---

# <center>Part 4.集成算法的训练与优化技巧

In [6]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

import warnings
warnings.filterwarnings('ignore')

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

# 自定义模块
from telcoFunc import *
# 导入特征衍生模块
import features_creation as fc
from features_creation import *

# re模块相关
import inspect, re

# 其他模块
from tqdm import tqdm
import gc

&emsp;&emsp;然后执行Part 1中的数据清洗相关工作：

In [7]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [8]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

&emsp;&emsp;同时，创建自然编码后的数据集以及经过时序特征衍生的数据集：

In [9]:
# 划分训练集和测试集
train, test = train_test_split(tcc, random_state=22)

X_train = train.drop(columns=[ID_col, target]).copy()
X_test = test.drop(columns=[ID_col, target]).copy()

y_train = train['Churn'].copy()
y_test = test['Churn'].copy()

X_train_seq = pd.DataFrame()
X_test_seq = pd.DataFrame()

# 年份衍生
X_train_seq['tenure_year'] = ((72 - X_train['tenure']) // 12) + 2014
X_test_seq['tenure_year'] = ((72 - X_test['tenure']) // 12) + 2014

# 月份衍生
X_train_seq['tenure_month'] = (72 - X_train['tenure']) % 12 + 1
X_test_seq['tenure_month'] = (72 - X_test['tenure']) % 12 + 1

# 季度衍生
X_train_seq['tenure_quarter'] = ((X_train_seq['tenure_month']-1) // 3) + 1
X_test_seq['tenure_quarter'] = ((X_test_seq['tenure_month']-1) // 3) + 1

# 独热编码
enc = preprocessing.OneHotEncoder()
enc.fit(X_train_seq)

seq_new = list(X_train_seq.columns)

# 创建带有列名称的独热编码之后的df
X_train_seq = pd.DataFrame(enc.transform(X_train_seq).toarray(), 
                           columns = cate_colName(enc, seq_new, drop=None))

X_test_seq = pd.DataFrame(enc.transform(X_test_seq).toarray(), 
                          columns = cate_colName(enc, seq_new, drop=None))

# 调整index
X_train_seq.index = X_train.index
X_test_seq.index = X_test.index

In [10]:
ord_enc = OrdinalEncoder()
ord_enc.fit(X_train[category_cols])

X_train_OE = pd.DataFrame(ord_enc.transform(X_train[category_cols]), columns=category_cols)
X_train_OE.index = X_train.index
X_train_OE = pd.concat([X_train_OE, X_train[numeric_cols]], axis=1)

X_test_OE = pd.DataFrame(ord_enc.transform(X_test[category_cols]), columns=category_cols)
X_test_OE.index = X_test.index
X_test_OE = pd.concat([X_test_OE, X_test[numeric_cols]], axis=1)

## <center>Ch.3 模型融合基础方法

&emsp;&emsp;本节开始，我们将开始介绍模型融合的相关内容，本节将重点介绍模型融合技术的核心概念与基本方法，下一小节开始将继续介绍如何借助这些方法进行模型优化。

&emsp;&emsp;需要注意的是，虽然模型融合的本质仍然是一种优化方法，但和我们此前接触到的算法与优化器有着本质上的区别，可以说我们在算法学习时，比较“痛苦”的是概念理解与理论推导，而算法实践过程相对简单，但模型融合正好相反，其理论并不复杂，但实践起来会有非常高的难度——或者说要能实践的有效果（比别人更好），难度是非常高的。因此在正式介绍模型融合的方法之前，我们需要从一个更为本质的角度理解到底什么是模型融合、其背后的核心思想是什么，才能更好的指导我们将这些方法活学活用。

> 注意，本部分内容学习需要在熟知Bagging原理的情况下进行。

## 一、认识模型融合

&emsp;&emsp;对于模型融合，相信很多同学都略有耳闻，作为一项诞生时间并不算久的新兴技术，在不到五年的时间内迅速成为当前机器学习领域提升模型效果最有效的方法之一，同时也是当前竞赛冲榜、企业实践必不可少的优化技术。那到底什么是模型融合呢？尽管截至目前并没有非常权威的严谨定义，但大家的普遍认同的一个观点是：只要是借助了多个模型的结果来完成最终预测，就可以算是模型融合。从这个角度来说，模型融合的概念其实是非常泛的，甚至是我们所熟知的集成学习，都可以看成模型融合的一类算法。因此，我们不妨从集成学习出发，认识到底什么是模型融合。

&emsp;&emsp;这里我们可以思考一个问题：有没有一种方法，能对集成算法进行集成？

&emsp;&emsp;在上一小节中，我们通过随机森林的超参数优化，获得了截至目前最好的预测结果。当然，就目前机器学习的应用实践效果来看，在大多数情况下，集成算法都是效果最好的一类算法。而所谓的集成算法，其实就是围绕一些分类性能相对较弱的基础分类器、通过某些方式来进行集成，最终的到一个分类性能更强的算法。其中最有效的集成策略就是Bagging和Boosting，前者通过并行的训练多个分类器，通过样本多样性和特征多样性训练多个“弱且不同”的基础分类器，再通过投票法或者均值法进行输出结果的集成；后者则通过串联的方式，通过不断提升误分类样本重要性权重来不断优化学习策略，最终逐渐提升模型最终输出结果。

<center><img src="https://s2.loli.net/2022/05/14/X4OIsyE2J1Uzalq.png" alt="image-20220514164956088" style="zoom:33%;" />

&emsp;&emsp;而更加“神奇”的是，通过这样的一个“集成”的过程，最终模型所表现出来的性能居然能远远高于参与集成的基础分类器性能，很明显，这肯定是“集成”这个过程的功劳，但这也不禁让人产生联想，既然多个模型的“集成”能够大幅提升建模效果，那围绕集成算法进一步进行“集成”，是否能进一步提高模型效果呢？

&emsp;&emsp;尽管哪怕作为初学者的我们也并不难理解这一想法的出发点，但要落地实践集成学习的集成其实并不简单，或者说基于这个思路的理论突破实践突破都需要一段时间。集成算法已成名近10年，而直到2018年之后，围绕集成算法的“集成”才初露锋芒，直至今日，围绕集成算法进行集成在理论端和应用端都有了不小的突破，其中理论层面的突破以周志华教授主导研究的深度森林方向研究成果最为显著，而实践应用层面的突破则以模型融合技术的广泛应用为有力证明。

### 1.算法集成的核心概念：集成方法、集成范式与集成算法

&emsp;&emsp;而为何会有理论层面和实践应用层面不同方向的技术划分？这就不得不提一组概念，帮助大家从一个更加本质的角度理解集成学习、同时也为本节将要介绍的模型融合方法做最基础的理论铺垫。

&emsp;&emsp;这组概念是：集成方法、集成范式与集成算法。

#### 1.1 集成方法

&emsp;&emsp;所谓集成方法，指的是把不同模型的输出结果融合为一个结果的过程，例如随机森林模型在输出最终结果时所采用的投票法、均值法等，几种简单的集成方法示例如下：

<center><img src="https://s2.loli.net/2022/05/08/6g9Ep3aeQ8XRYTK.png" alt="image-20220508221756193" style="zoom:33%;" />

当然，根据周志华教授的理论，集成方法也被称作结合方法。大多数集成方法的诞生时间都比较早，约在上世纪九十年代初就已经成型，也是构成集成算法最底层的技术体系。需要知道的是，集成方法是有一整套完整理论基础的方法，根据Dietterich等人的表述，集成带来的益处可以总结为以下三点：（摘自《集成学习：基础与算法》）

- 统计方面：通常情况下，假设空间会非常大，基于有限的训练集无法进行有效的探索；有些情况下，甚至会有多个不同的假设在训练集上取得相同的准确率。如果学习算法从中选取一个，一旦误选，就会产生无法很好预测未知数据的风险。采用集成方法能够很好的结合这些假设，从而降低错选假设的风险；

- 计算方面：许多学习算法在搜索时会陷入局部最优解。即便有足够多的训练数据，寻找最优解仍然是一件困难的事情。集成方法从多个不同的起点进行搜索，可以更大概率寻找到一个更优的解；

- 表示能力方面：在很多机器学习任务中，潜在的真是假设不能被假设空间的任一假设所表示。通过集成方法，能够结合多种假设，可以拓展假设空间，因此，学习算就可能得到对真实未知假设更精准的近似。

总结来说，对模型进行有效的集成能大幅提升模型判别效力，同时降低方差和偏差带来的影响。当然怎样的集成才是有效的集成，会在本部分内容后期进行探讨。

> 现今大多数流行的模型融合方法本质上都是集成方法或者结合方法，例如Stacking方法，则是在1992年由Wolpert等人提出的一种结合方法。Wolpert, David H. “Stacked generalization.” Neural networks 5.2 (1992): 241-259.

#### 1.2 集成范式

&emsp;&emsp;而所谓的集成范式，其实指的是不同模型的“组合”方法，例如Bagging是一种类似于并联的模型组合方法，而Boosting则是一种类似于串联的模型组合方法，集成范式规定了待集成的这些模型的基本训练方法，是拥有完整理论体系的复杂方法，任何集成过程只要能满足这类范式的理论条件，都能达到集成的理论效果，换而言之，集成范式是保证集成效果的理论依据，而Bagging和Boosting也是目前最通用也最著名的两种集成范式。

#### 1.3 集成算法

&emsp;&emsp;而集成算法，简单理解其实就是集成范式+集成方法的结果，例如分类随机森林，实际上就是Bagging（集成范式）+投票法（集成方法）。当然集成范式只有一个理论层面可行的模型训练思路，在集成算法落地的过程中还需要设计很多关键环节，如如何确保分类器独立性等，这也就是基于一个集成范式可能诞生多个不同集成算法的原因。

### 2.模型融合与深度森林

&emsp;&emsp;在了解了集成方法、集成范式与集成算法之间的关系之后，接下来我们再来看集成算法的集成技术在当前的两个不同突破方向。

- 模型融合

&emsp;&emsp;首先我们知道，机器学习算法实际上是一种“后验”的算法：有些操作可以没有太强的理论依据，但只要在测试集上表现出了不错的效果、甚至是在交叉验证过程中表现出了不错的效果，那么这个模型就是可以用的“好模型”。在这个思路指导下，机器学习领域尝试新方法的门槛变得很低，机器学习的建模流程也随之变得相对比较灵活，很多方法可以先试试看，“有效果就用，没效果就算”。很明显，这就给了前端技术应用人员很大的创新空间。

&emsp;&emsp;因此，在集成算法诞生了一段时间之后，偏向实践应用端的算法工程人员开始尝试集成算法+集成方法的“集成”策略，这也就是后来被人们称为模型融合的一类方法，尽管前端偏应用的算法工程师们并不打算像科研工作者一样先从理论层面进行论证与突破，但这种应用层面的创新，在机器学习整体属于后验算法的有力支持下，也是取得了重大的突破。

&emsp;&emsp;模型融合的背后的底层思想非常简单：先训练若干个集成算法，然后围绕这些集成算法的预测结果来进行集成，例如投票法、平均法等都是模型融合常用的集成方法。例如一个简单的基于随机森林、XGBoost、和LightGBM三个模型的模型融合过程如下：

<center><img src="https://s2.loli.net/2022/05/16/XHhyeTpr4ZAUwlM.png" alt="image-20220516171447583" style="zoom:33%;" />

&emsp;&emsp;不过需要注意的是，尽管集成算法+集成方法的思路并不复杂，但缺少了集成范式的支持，最终模型融合的效果其实是很难保障的。例如，在大多数情况下随机森林效果都要好于决策树，但对于投票法的模型融合，我们很难说三个集成算法的投票结果就一定好于单独一个集成算法的预测结果。

&emsp;&emsp;要如何解决这个问题，熟知Bagging和Boosting等集成范式的算法工程师们决定借鉴这些范式的基本思路，例如Bagging要求基础分类器尽可能的存在差异性，差异越明显最终结果越好（最好彼此之间相互独立），因此如果是要通过投票的方式进行模型融合，则可以尝试尽可能让参与融合的这些算法各自差异性更加明显一些，例如通过构造样本多样性、特征多样性、算法多样性等方法来达成这一目的。再比如，Boosting通过串联的方式让模型在不同阶段侧重学习数据的部分成分，甚至可以通过层级设计，让模型逐步拟合误差，以提高模型效果，受此启发算法工程师们决定采用最早诞生于1992年的Stacking方法，分层设计融合策略——即上一层输出的结果可以作为下一层的训练数据，通过类似拟合误差的方法来获得一个更好的预测效果；同时，经过一段时间的实践人们发现，由于集成学习本身极强的学习能力，模型融合过程极容易过拟合，因此诞生了Stacking的改良型算法——Blending。等等等等，诸如此类，不一而足。

&emsp;&emsp;需要注意的是，正是因为模型融合方法都是一系列基于实践探索出来的方法，因此没有“一定行之有效”的“标准流程”，很多时候如何进行融合、如何通过融合获得一个更好的效果，都需要算法工作人员反复尝试、不断探索才能获得。并且，正是因为模型融合中集成范式的缺失，导致往往需要“启发式”的对融合的关键环节进行修改，例如加权融合时的权重是否可以视作超参数进行搜索、单独一个模型是否可以通过构建样本多样性来进行模型融合等等，如何活学活用这些机器学习方法，也是模型融合过程需要克服的不小难题。

&emsp;&emsp;正因如此，模型融合的课程内容将主要分为两方面，其一当然是模型融合中常用的集成方法的介绍，例如投票法、平均法、Stacking、Blending方法介绍；其二则是重点介绍这些方法如何应用，以及如何针对当前数据集情况设计一个行之有效的融合流程，并且，机械的方法调用将只占一小部分，我们将分配更多的篇幅和时间来介绍一些启发式的融合方法，学以致用、活学活用，才是模型融合方法学习的最终目的。

- 深度森林

&emsp;&emsp;如果说偏实践应用的算法工程人员探索出了模型融合方法，那么偏理论的算法学术工作者则探索了一套拥有完整集成范式理论基础的集成算法的集成方法，其中最有名的算法就是周志华教授提出的[深度森林（Deep Forest， 2010）](https://arxiv.org/pdf/1702.08835.pdf)。该算法的前生是在周志华教授2018提出的gcForest（多粒度级联森林）算法，通过类DNN的层级架构来提升模型复杂度、并且通过集成随机森林（以及完全随机树森林）来提升模型效果，其基本集成范式（算法架构）如下所示：

<center><img src="https://s2.loli.net/2022/05/14/T1jhiCVox5gWv6J.png" alt="image-20220514205154541" style="zoom:50%;" />

&emsp;&emsp;在这个基本架构下，每一层接收到的数据都是上一层的预测输出结果以及原始特征（或者原始特征的一部分），而输出的则是这一层所有随机森林的预测结果。并且支持根据特征相关性对其进行分组，然后不同组输入到不同层进行训练，以最大化利用特征中提供的有效信息。可以说，深度森林是目前“基于集成算法的集成算法”中最著名也是表现最好的一个，同时也是自2017年CatBoost发表之后近五年来最值得期待的集成算法。

&emsp;&emsp;尽管深度森林从算法流程上来看好像也就是模型融合的“进阶版”——更多集成算法+更复杂的融合流程，但实际上这个流程拥有相比模型融合更加扎实的理论基础——即该流程的实际建模效果是可以通过理论进行论证的，并且拥有更加完整和严谨的超参数体系，而不像其他融合方法的使用更依赖带入实际数据的测试结果。当然，这也就是此前我们介绍的集成范式起到的作用。

&emsp;&emsp;不过需要注意的是，尽管深度森林是拥有完整集成范式的基于集成算法的集成算法，但对于实际预测建模来说，深度森林也只是我们可选的算法之一，并不是集成算法在进行集成时的最终形态，也无法代替模型融合方法在实际建模预测中的作用。因此，对于算法工程人员来说，从算法掌握层面来说，我们应当快速学习新兴算法，从算法实践层面来说，我们也应当熟练掌握各类模型融合技巧，并在实践中不断积累模型融合方法使用经验。

> 在学术界，也有学者认为深度森林应该属于深度学习范畴，这里不做学术讨论，仅通过对比帮助大家加深模型融合等相关概念的理解。

> 知识点补充：关于基于结果的模型融合与基于过程的模型融合概念辨析      
&emsp;&emsp;其实通过上面的介绍我们不难发现，模型融合和集成算法的概念是高度重叠的，因此除了可以从集成学习的角度理解模型融合外，也可以从模型融合的角度去理解集成学习。        
&emsp;&emsp;有一种观点是，从模型融合的大类上来划分，其实是可以分为基于过程的模型融合和基于结果的模型融合：所谓的集成算法其实都是基于结果的模型融合，这类方法会根据最终输出结果来回调参与融合（或者集成）的基础模型超参数，例如随机森林在训练时为了提高泛化能力，往往会对参与集成的决策树进行剪枝操作，而深度森林其实也会根据最终结果来调整参与集成的每个森林的超参数；而所谓的基于结果的模型融合，则会在模型已经给出最终预测结果的情况下，更多的考虑这些结果应该如何融合，也就是前面所说的集成方法的使用，并围绕集成方法来进行优化和调整，例如如下投票法的调整过程：<center><img src="https://s2.loli.net/2022/05/08/6g9Ep3aeQ8XRYTK.png" alt="image-20220508221756193" style="zoom:33%;" /></center>            
&emsp;&emsp;当然，无论是基于过程的融合还是基于结果的融合，都只是学术概念上的讨论与划分，只需要了解即可，不必深究。

### 3.模型融合的技术特点

&emsp;&emsp;在了解了模型融合技术诞生背景和技术发展底层逻辑之后，接下来我们简单总结模型融合的技术特点。

- 模型优化的终极武器

&emsp;&emsp;很多时候，在大家普遍使用这些通用的集成算法+优化器，甚至是采用相同的超参数优化策略，模型最终得分基本也是类似的，此时能够进一步提高模型效果的措施自然而然就成了决定胜败的关键性因素，其中模型融合与特征衍生，就是能进一步提高模型表现的终极武器。并且模型融合和特征衍生相同，都是方法众多且极度灵活的优化策略，并没有一个一定有效或者一定能获得最优结果的通用流程，如何制定融合策略和衍生策略，将会是极度考验算法工程师个人能力的事项。

> 不过这里也需要注意的是，模型融合方法能起到决定性效果的前提，是有能力先训练出一批效果极好的模型算法，然后用模型融合方法在一个比较好的结果上再进行提升，当然，这样的提升也才是有意义的。反之，如果无法到达能够训练好一批模型算法（尤其是集成算法）的程度，直接使用一些未经优化、效果一般的模型进行融合，哪怕融合能帮模型提升效果，但可能优化的结果甚至还达不到一个超参数优化较好的集成算法。因此模型融合更像是高手过招的终极武器，但却无法在基础不牢的新人手上发挥作用。

- 方法原理简单且种类众繁多，方法实践困难

&emsp;&emsp;正如开篇所言，模型融合其实方法原理并不复杂，投票法平均法自不必说，哪怕是Staking和Blending这类复杂的融合方法，可能就原理层面来说也不如决策树或者逻辑回归复杂，但和特征衍生类似，模型融合的方法种类较为繁多，因此本节我们会设置完整的一部分内容先帮大家梳理和整理目前较为通用的模型融合方法，并在这个过程中配套定义相关的使用函数。

&emsp;&emsp;但是，模型融合的实践过程会非常复杂，这里的复杂并不是指调用这些方法的过程复杂，而是获得一个比较好的融合结果会比较困难，这里会需要介绍更多的实战层面的技巧，以及同时融入特征衍生和筛选的内容进行同步的实践，这部分内容我们将在下一小节进行介绍。

- 创造多样性往往是融合成败的决定性因素

&emsp;&emsp;最后，我们再补充一点更深度的模型融合的技术特点——多样性问题。对于模型融合来说，多样性构建的越好，最终融合结果也往往会更好。

&emsp;&emsp;这里的构建多样性有几个方面含义，包括样本多样性（带入不同样本训练不同模型）、特征多样性（带入不同特征训练不同模型）、模型多样性（融合时采用多个不同模型进行融合）等，通过提升多样性来提升融合效果，这其实就是Bagging集成的思路——往往我们需要参与融合的这些算法“和而不同”，即每个算法最好是彼此之间存在一定的独立性（最好是完全相互独立，但不可能做到），根据Bagging的理论，在此情况下多个模型能够更好得互补，自然而然在进行融合时，就能得到一个更好的结果。

&emsp;&emsp;不过关于如何构造多样性来提高融合效果，这会是一个比较进阶的问题，例如如何训练多个和而不同的集成算法、如何通过特征衍生来构建特征多样性、如何先进行单独模型融合（只包含一类、多个模型的融合）再进行多模型分层融合等等，这些内容我们将同一放在下一节进行介绍，本节将先采用此前优化后得到的逻辑回归、决策树（Part 2）以及随机森林（Part 4）来进行模型融合的方法层面教学。

- 模型创建

&emsp;&emsp;接下来再次创建此前已经超参数优化后的模型，其中逻辑回归和决策树我们在Part 2中就已经搜索得到了一组最优超参数，此处我们再次导入相关模型，由于Part 2中的逻辑回归是自定义的评估器，并且和数据预处理流程共同构建了一个机器学习流，最终带入网格搜索进行的优化，这里我们简单复习该过程如下：

In [11]:
class logit_threshold(BaseEstimator, ClassifierMixin, TransformerMixin):
    
    def __init__(self, penalty='l2', C=1.0, max_iter=1e8, solver='lbfgs', l1_ratio=None, class_weight=None, thr=0.5):
        self.penalty = penalty
        self.C = C
        self.max_iter = max_iter
        self.solver = solver
        self.l1_ratio = l1_ratio
        self.thr = thr
        self.class_weight = class_weight
        
    def fit(self, X, y):
        clf = LogisticRegression(penalty = self.penalty, 
                                 C = self.C, 
                                 solver = self.solver, 
                                 l1_ratio = self.l1_ratio,
                                 class_weight=self.class_weight, 
                                 max_iter=self.max_iter, 
                                 random_state=12)
        clf.fit(X, y)
        self.coef_ = clf.coef_
        self.clf = clf
        return self
        
    def predict_proba(self, X):
        res_proba = self.clf.predict_proba(X)
        return res_proba
    
    def predict(self, X):
        res = (self.clf.predict_proba(X)[:, 1]>=self.thr) * 1
        return res

这里对Part 2中的logit_threshold进行了修改，其一是增加了predict_proba方法，使其能够对结果概率进行预测，其二则是增加了一个父类ClassifierMixin，相当于标注该评估器是一个Classifier，为了后续输入投票法评估器做准备。该评估器改写后，需要替换telcoFunc.py文件中原始版本，方便后续直接调用。

In [12]:
# 设置转化器流
logistic_pre = ColumnTransformer([
    ('cat', preprocessing.OneHotEncoder(drop='if_binary'), category_cols), 
    ('num', 'passthrough', numeric_cols)
])

num_pre = ['passthrough', preprocessing.StandardScaler(), preprocessing.KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='kmeans')]

# 实例化逻辑回归评估器
logistic_model = logit_threshold(max_iter=int(1e8))

# 设置机器学习流
logistic_pipe = make_pipeline(logistic_pre, logistic_model)

# 设置超参数空间
cw_l = [None, 'balanced']
#cw_l.extend([{1: x} for x in np.arange(1, 4, 0.2)])
logistic_param = [
    {'columntransformer__num':num_pre, 'logit_threshold__thr': np.arange(0.1, 1, 0.1).tolist(), 'logit_threshold__penalty': ['l1'], 'logit_threshold__C': np.arange(0.1, 1.1, 0.1).tolist(), 'logit_threshold__solver': ['saga'], 'logit_threshold__class_weight':cw_l}, 
    {'columntransformer__num':num_pre, 'logit_threshold__thr': np.arange(0.1, 1, 0.1).tolist(), 'logit_threshold__penalty': ['l2'], 'logit_threshold__C': np.arange(0.1, 1.1, 0.1).tolist(), 'logit_threshold__solver': ['lbfgs', 'newton-cg', 'sag', 'saga'], 'logit_threshold__class_weight':cw_l}, 
]

# 实例化网格搜索评估器
logistic_search = GridSearchCV(estimator = logistic_pipe,
                               param_grid = logistic_param,
                               scoring='accuracy',
                               n_jobs = 15)

s = time.time()
logistic_search.fit(X_train_OE, y_train)
print(time.time()-s, "s")

549.1211280822754 s


In [13]:
logistic_search.best_score_

0.8104888764656977

In [14]:
logistic_search.score(X_train_OE, y_train), logistic_search.score(X_test_OE, y_test)

(0.8123816736084817, 0.7836456558773425)

&emsp;&emsp;接下来是决策树模型的创建过程，这里我们直接带入Part 2中决策树最优参数进行模型实例化与模型训练：

In [15]:
tree_model = DecisionTreeClassifier(ccp_alpha=0, 
                                    max_depth=5, 
                                    max_leaf_nodes=8, 
                                    min_samples_leaf=1, 
                                    min_samples_split=2, 
                                    random_state=12).fit(X_train_OE, y_train)

In [16]:
tree_model.score(X_train_OE, y_train), tree_model.score(X_test_OE, y_test)

(0.7991291177584249, 0.768313458262351)

&emsp;&emsp;最后是随即森林模型创建过程，这里我们直接带入上一小节中随机森林最优参数进行模型实例化与模型训练：

In [17]:
RF_0 = RandomForestClassifier(max_depth = 10, 
                              max_features = 'sqrt',
                              max_leaf_nodes = None, 
                              max_samples = 2000, 
                              min_samples_leaf = 2, 
                              min_samples_split = 7, 
                              n_estimators = 97,
                              random_state=12).fit(X_train_OE, y_train)

In [18]:
RF_0.score(X_train_OE, y_train), RF_0.score(X_test_OE, y_test)

(0.8483528966300644, 0.7955706984667802)

&emsp;&emsp;能够看出，从测试集表现来看，随机森林效果还是要明显好于逻辑回归与决策树模型。

- 模型本地保存

&emsp;&emsp;考虑到在后续的模型融合方法介绍中还要反复用到这些模型，为了调用方面，这里我们可以通过joblib的方式进行模型的本地保存。首先需要导入相关的功能模块，其中dump主要用于本地模型保存，而load则负责本地模型读取：

In [19]:
from joblib import dump, load

> 注，除了joblib外还可使用pickle进行模型保存。不过根据sklearn官网的相关介绍，相比pickle，joblib会更适合保存机器学习中包含大量array数组的对象。

> joblib是anaconda附带的安装包，若是使用anaconda进行python安装，则直接导入即可，若当前环境中没有joblib，则可以使用pip install joblib方法进行安装。

然后可以使用dump函数进行模型保存：

In [20]:
dump(logistic_search, 'logistic_search.joblib') 
dump(tree_model, 'tree_model.joblib') 
dump(RF_0, 'RF_0.joblib') 

['RF_0.joblib']

然后即可在当前操作空间的主目录下看到三个joblib文件。然后我们可以通过load的方式进行读取：

In [21]:
logistic_search = load('logistic_search.joblib') 
tree_model = load('tree_model.joblib') 
RF_0 = load('RF_0.joblib') 

In [22]:
logistic_search

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('cat',
                                                                         OneHotEncoder(drop='if_binary'),
                                                                         ['gender',
                                                                          'SeniorCitizen',
                                                                          'Partner',
                                                                          'Dependents',
                                                                          'PhoneService',
                                                                          'MultipleLines',
                                                                          'InternetService',
                                                                          'OnlineSecurity',
                                       

这里需要主要的是，此处保存的三个评估器，其中logistic_search是网格搜索评估器，而剩下两个则是模型评估器，在调用的过程中需要注意二者区别。

&emsp;&emsp;在准备好了模型之后，接下来正式进入模型融合的方法介绍中。